In [15]:
import csv
import numpy as np
from collections import Counter
import random
import time
import statistics
import itertools

In [16]:
file = open("C:\\Users\\Simeon Horner\\Documents\\GitHub\\RiskAndLogisticsA2\\Data Files\\CSV versions\\OrderList.csv", encoding="utf-8")
csvreader = csv.reader(file)

Orders = []

for row in csvreader:
    Orders.append([int(x) for x in row[1:]])

Distances = np.genfromtxt('C:\\Users\\Simeon Horner\\Documents\\GitHub\\RiskAndLogisticsA2\\Data Files\\CSV versions\\DistanceMatrix_meters.csv', delimiter=',', skip_header=0, dtype='i')
Distances[0, 0] = 0

In [17]:
def ConstructionHeuristic_random():
    newShelves = [0]*96
    currShelf = 0

    tempProducts = [x for x in range(1, 91)]

    for i in range(len(newShelves)):
        random_prod = random.choice(tempProducts)
        tempProducts.remove(random_prod)
        newShelves[currShelf] = random_prod
        currShelf += 1

        if len(tempProducts) == 0:
            tempProducts = [x for x in range(1, 91)]

    return newShelves




In [18]:
def find_closest_product(current_shelf, order, distances):
    closest_shelf = None
    min_distance = float(1e6)
    for shelf in order:
        if distances[current_shelf][shelf] <= min_distance:
            closest_shelf = shelf
            min_distance = distances[current_shelf][shelf]
    return closest_shelf

def generate_order_lists(order):
    ''' 
    Takes a list with tuple elements and returns a list of lists 
    with all possible combinations of individual elements individual elements.
    '''
    order_lists = []
    tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
    for combination in itertools.product(*[order[i] for i in tuple_indices]):
        new_order = order.copy()
        for i, index in enumerate(tuple_indices):
            new_order[index] = combination[i]
        order_lists.append(new_order)
    return order_lists

def greedy_order_route(order, distances):
    ''' 
    Uses a greedy method of calculating the minimum distance. 

    Function has been split into two if statements to consider
    cases of orders where products are contained on more than one shelf.

    If products are contained on more than one shelf, the function constructs 
    a route with all possible shelf combinations and chooses the one with 
    the shortest distance.

    Returns a list containing the route and the total distance for the order.
    '''

    # If all products in the order are contained on one shelf only
    if not any(isinstance(product, tuple) for product in order):
        visited = [0]
        current_position = 0  
        for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
        visited.append(0)
        OrderDistance = 0
        for i in range(len(visited) - 1):
            OrderDistance += distances[visited[i]][visited[i+1]]
        order_distance_final = OrderDistance
        visited_final = visited

    # If one or more products in the order are contained on more than one shelf
    elif any(isinstance(product, tuple) for product in order):
        order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
        order_routes = []   # initialise a list of routes for all combinations                                 
        order_distances = [] # initialise a list of distances for all combinations
        
        # loop over all combinations
        for order in order_combinations:
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
            visited.append(0)
            order_routes.append(visited) # add the route for the combination to the list of routes
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
        # select the order with the shortest distance among the combinations
        min_idx = order_distances.index(min(order_distances))
        visited_final = order_routes[min_idx] 
        order_distance_final = order_distances[min_idx]
            
    return visited_final, order_distance_final # return order route and distance

def convert_orders_to_shelf_indices(allocations):
    ''' 
    This function takes the allocation vector and returns an 
    order matrix with shelf indices instead of product indices.
    '''
    product_to_shelf = {}
    for shelf_index, product in enumerate(allocations):
        if product != 0:  # Check if the element is not zero
            if product not in product_to_shelf:
                product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
            else:
                # If the product already exists in the dictionary, append the new shelf index to the list
                product_to_shelf[product].append(shelf_index + 1)

    # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
    product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

    OrdersByShelf = []
    for order in Orders:
        order_shelf_indices = []
        for product_index in order:
            if product_index in product_to_shelf_tuples:
                shelf_indices = product_to_shelf_tuples[product_index]
                order_shelf_indices.append(shelf_indices)
            else:
                order_shelf_indices.append(0)  # Product not found in allocation matrix
        OrdersByShelf.append(order_shelf_indices)

    return OrdersByShelf



def q1_function(allocation_vector, distance_matrix):

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, distance_matrix)
        routes.append(visited_order_route)
        DistancesPerOrder.append(visited_order_dist)
        TotalDistance += visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)

    # List of routes ordered by distance (descending)
    OrderedRoutes = [routes[i] for i in idx_longest_orders]
    
    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes

In [21]:
times = []
scores = []


for i in range(100):
    start = time.time()
    Shelves = ConstructionHeuristic_random()
    end = time.time()
    times.append(end-start)

    scores.append(q1_function(Shelves, Distances)[0])

In [22]:
print(statistics.mean(times))
print(statistics.mean(scores))

9.242534637451172e-05
278901


In [23]:
CurrentAllocation = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41, 2, 46, 60, 67, 56, 83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86, 3, 17, 90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47, 6, 30, 21, 43, 64, 66, 78, 76, 61, 8, 72, 22, 18, 82, 14, 28, 4, 5, 84, 54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50, 7, 88, 9, 40, 77, 15, 1, 33, 59, 42, 52, 0, 0, 0, 0, 0, 0]

In [24]:
print(q1_function(CurrentAllocation, Distances)[0])

286860


In [26]:
print(ConstructionHeuristic_random())

[90, 6, 85, 41, 17, 71, 54, 61, 63, 79, 16, 75, 69, 40, 60, 65, 22, 70, 59, 88, 77, 44, 2, 21, 33, 24, 35, 74, 72, 27, 46, 15, 12, 30, 86, 39, 29, 45, 7, 81, 66, 32, 5, 89, 1, 84, 28, 8, 57, 31, 80, 37, 64, 56, 36, 51, 73, 43, 20, 26, 11, 10, 76, 9, 25, 23, 18, 19, 13, 47, 4, 87, 48, 62, 14, 38, 83, 50, 67, 53, 82, 52, 78, 58, 3, 55, 34, 42, 68, 49, 69, 85, 47, 4, 90, 34]
